# SPR 2026 - TF-IDF + SGDClassifier

**RÁPIDO: SGDClassifier é otimizado para dados esparsos de alta dimensão**

Vantagens:
- Treinamento incremental (stochastic gradient descent)
- Escala muito bem com número de features
- Suporta regularização L1/L2/ElasticNet
- Tempo de execução: **< 1 minuto**

---
**CONFIGURAÇÃO OFFLINE:**
1. No Kaggle, Settings → Internet → **OFF**
---

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.calibration import CalibratedClassifierCV
import warnings
warnings.filterwarnings('ignore')

SEED = 42
DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'

np.random.seed(SEED)
print('Bibliotecas carregadas!')

In [ ]:
# Carregar dados
train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')

print(f'Train: {train.shape}')
print(f'Test: {test.shape}')

In [ ]:
# TF-IDF
tfidf = TfidfVectorizer(
    max_features=15000,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.95,
    sublinear_tf=True
)

X_train = tfidf.fit_transform(train['report'])
X_test = tfidf.transform(test['report'])
y_train = train['target'].values

print(f'TF-IDF shape: {X_train.shape}')

In [ ]:
# SGDClassifier com loss 'log_loss' (equivalente a Logistic Regression)
# mas muito mais rápido para dados esparsos
model = SGDClassifier(
    loss='log_loss',        # Log loss para classificação probabilística
    penalty='l2',           # Regularização L2
    alpha=1e-4,             # Força da regularização
    max_iter=1000,
    tol=1e-3,
    class_weight='balanced',
    random_state=SEED,
    n_jobs=-1,
    early_stopping=True,
    validation_fraction=0.1
)

print('Treinando SGDClassifier...')
model.fit(X_train, y_train)
print('Modelo treinado!')

In [ ]:
# Predições e submissão
predictions = model.predict(X_test)

submission = pd.DataFrame({
    'ID': test['ID'],
    'target': predictions
})

submission.to_csv('submission.csv', index=False)

print('submission.csv criado!')
print(submission['target'].value_counts().sort_index())